In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from . import jupyter
from .studioapp import start

%gui studio

ModuleNotFoundError: No module named '__main__.studioapp'; '__main__' is not a package

In [4]:
start("nowy")

NameError: name 'start' is not defined

In [42]:
shower=None
import gc
gc.collect()

0

In [37]:
import cv2
import numpy as np
from reactive.var import Var
import helpers

import os
import gc
import asyncio

imgvar = Var(np.zeros((1,1)))
shower = helpers.show_image('hejha', imgvar)
#asyncio.ensure_future(shower.dispose())
gc.collect()
#os.write(1, b"aaa\n")
#imgvar.set(cv2.imread("lena.png"))


deleting <reactive.var.Var object at 0x7f3fd13fe0f0> False None
deleting <reactive.var.Var object at 0x7f3fd13fe7f0> False <function ensure_coro_func.<locals>.async_f at 0x7f3fd14f82f0>


338

In [36]:
shower.disposed

False

In [38]:
imgvar.set(cv2.imread("flower.jpg"))
gc.collect()

338

In [36]:

imgvar.set(cv2.imread("lena.png"))


In [28]:
import studioapp

window = studioapp.default_main_window
ii = window.widgets['wykresik'].widget.axes.imshow(imgvar.data)
window.widgets['wykresik'].widget.draw()

In [12]:
import asyncio
asyncio.ensure_future(shower.dispose())

<Task pending coro=<Var.dispose() running at /home/peper/src/databrowser/reactive/var.py:94>>

In [19]:
del shower

In [20]:

import gc

gc.collect()

281

In [16]:
async def plot(data: dict, plot_brick=None):
    plot_line, = plot_brick.axes.plot(data)
    plot_brick.draw()
        
    async def cleanup(self):
        plot_line.remove()

dir(plot)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [26]:
plot.__dict__

{}

In [6]:
import reactive
import asyncio


def db(a):
    return a*2


db2 = reactive.unroll_gen(0)(db)

async def myrange(n):
    for i in range(n):
        yield i

@reactive.reactive2()
async def add(a, b):
    return a+b

add2 = reactive.unroll_gen('a')(add)
gen = None
async def main():
    global gen
    r = await add(5, 4)
    print(r.data)
    
    gen = add2(a=myrange(4), b=5)
    print("is: %s" % (await gen.__aiter__().__anext__()).data)
    #async for i in gen:
    #    print(i)
        
asyncio.get_event_loop().run_until_complete(main())

9
is: ab


In [34]:
gen.__aiter__().__anext__().__await__().__next__()

StopIteration: <coroutine object wrapped_f at 0x7fbc767f2f10>

In [1]:
import asyncio

def deco(f):
    async def wrapped_f(a):
        print("przed")
        await f(a)
        print("po")
    return wrapped_f


async def foo(a):
    print(a)
    
f = foo(44)
loop = asyncio.get_event_loop()
loop.run_until_complete(f)

44


In [47]:
loop.run_until_complete(next(gen().__aiter__()))



TypeError: 'async_generator' object is not an iterator

In [56]:
dfrom collections import deque

async def gen():
    for i in range(10):
        for j in range(3):
            yield j,i  
            

class Tee:
    def __init__(self, gen):
        self.gen = gen.__aiter__()
        self.outputs = 0
        self.queue = deque()
        self.queue_index_offset = 0
        self.finished = False
        self.fetch_idle = asyncio.Event()
        self.fetch_idle.set()
        
    def ref(self):
        self.outputs += 1
        
    def unref(self, next_index):
        self.outputs -= 1
        assert self.outputs >= 0
        for i in range(next_index-self.queue_index_offset, len(self.queue)):
            self.queue[i][1]-=1

    def request_out(self):
        return TeeOut(self)
    
    async def pull_item(self, index):
        assert index >= self.queue_index_offset, "requested too old item; it's a bug"
        assert index <= self.next_index(), "non-sequential getter?"
        if index == self.next_index():
            if not await self._fetch_next():
                raise StopAsyncIteration
    
        assert index < self.next_index(), "fetch failed?"
            
        index_in_queue = index - self.queue_index_offset
        self.queue[index_in_queue][1] -= 1
        print("returning")
        return self.queue[index_in_queue][0]

    async def _fetch_next(self):
        if not self.finished:
            print("fetching")
            if self.fetch_idle.is_set():
                print("really fetching")
                self.fetch_idle.clear()
                try:
                    self.queue.append([await self.gen.__anext__(), self.outputs])
                    print("fetched")
                except StopAsyncIteration:
                    self.finished = True
                self.fetch_idle.set()
            else:
                #someone other is already fething
                await self.fetch_idle.wait()
        return not self.finished
                
    
    def next_index(self):
        return self.queue_index_offset + len(self.queue)
        
        
class TeeOut:
    def __init__(self, tee: Tee):
        self.tee = tee
        tee.ref()
        self.next_index = tee.next_index()
        
    def __del__(self):
        self.tee.unref(self.next_index)
    
    def __aiter__(self):
        return self
    
    async def __anext__(self):
        item = await self.tee.pull_item(self.next_index)
        self.next_index += 1
        return item


    
async def main():
    tee = Tee(gen())
    out = tee.request_out()
    print("hej")
    async for i in out:
        print("fd")
        print(i)
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

hej
fetching
really fetching
fetched
returning
fd
(0, 0)
fetching
really fetching
fetched
returning
fd
(1, 0)
fetching
really fetching
fetched
returning
fd
(2, 0)
fetching
really fetching
fetched
returning
fd
(0, 1)
fetching
really fetching
fetched
returning
fd
(1, 1)
fetching
really fetching
fetched
returning
fd
(2, 1)
fetching
really fetching
fetched
returning
fd
(0, 2)
fetching
really fetching
fetched
returning
fd
(1, 2)
fetching
really fetching
fetched
returning
fd
(2, 2)
fetching
really fetching
fetched
returning
fd
(0, 3)
fetching
really fetching
fetched
returning
fd
(1, 3)
fetching
really fetching
fetched
returning
fd
(2, 3)
fetching
really fetching
fetched
returning
fd
(0, 4)
fetching
really fetching
fetched
returning
fd
(1, 4)
fetching
really fetching
fetched
returning
fd
(2, 4)
fetching
really fetching
fetched
returning
fd
(0, 5)
fetching
really fetching
fetched
returning
fd
(1, 5)
fetching
really fetching
fetched
returning
fd
(2, 5)
fetching
really fetching
fetched
returning

In [ ]:
from ipyleaflet import Map

Map(center=[34.6252978589571, -77.34580993652344], zoom=10)

In [4]:
from ipywidgets import widgets
from IPython.display import display
button = widgets.Button(description="regrgreff")
display(button)
button.on_click(lambda a: print("heja"))
sli = widgets.IntSlider()
display(sli)

In [13]:
button2 = widgets.Button(description="regrgre")
display(button2)
button2.on_click(lambda a: print("heja2"))
display(button)

In [15]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
%matplotlib qt
import matplotlib.pyplot as plt
plt.plot([1,2,4])

In [1]:
import cpp.example as ex
%load_ext autoreload
%autoreload

In [2]:
import pandas as pd
import numpy as np
n = np.random.randn(5)
s = pd.Series(n, index=['a', 'b', 'c', 'd', 'e'])

In [6]:
s.values[1]=33

In [7]:
s

a    -1.143862
b    33.000000
c    -0.912464
d    -0.833601
e     0.458526
dtype: float64

In [8]:
ex.hej(s.values)

'd'

In [17]:
foo = ex.Foo()
ex.fooize3(foo)
foo.text

'hej2'

In [ ]:
%gui qt
import b
window = b.MainWindow()

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
#%gui qt
import sip
sip.setapi("QString", 2)
sip.setapi("QVariant", 2)
from PyQt5.QtGui  import *
from PyQt5.QtCore  import *
from PyQt5  import QtCore
from PyQt5.QtWidgets  import *
%gui qt

heja3


In [15]:
from studio import *
studio = Studio()
window = MainWindow()
window.show()

In [16]:
brick = PlotBrick(studio, window)
window.addDockWidget(Qt.LeftDockWidgetArea, brick)
data = studio.get_data()
brick.axes.plot(data["test"])

In [48]:
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

brick.mpl_toolbar = NavigationToolbar(brick.canvas, brick)